In [1]:
import stardog
from SPARQLWrapper import SPARQLWrapper, JSON, XML
from pymantic import sparql
from datetime import  datetime
import numpy as np
import pandas as pd

In [2]:
connection_details = {
  'endpoint': 'http://localhost:5820',
  'username': 'admin',
  'password': 'admin'
}

In [1]:
with stardog.Admin(**connection_details) as admin:
    for db in admin.databases():
        print(db.name)

In [4]:
database_name='RDF_based_approach_1'
Time_Dictionary=dict()
average_dict=dict()
stdv_dic=dict()

# Defining Function

In [5]:
def calculate_time(query,variable_name):
    start_time = datetime.now()
    conn.select(query, timeout=0)
    end_time = datetime.now()
    time=(round((end_time- start_time ).total_seconds()*1000))
    add_to_list(variable_name,time)   

    
def add_to_list(variable_name,time):
    if variable_name in Time_Dictionary:
        Time_Dictionary[variable_name].append(time)
    else:
        Time_Dictionary.update({variable_name:[]})
        Time_Dictionary[variable_name].append(time)

def change_name_index(value):
    value=value+1
    result= 'round '+ str(value)
    return result

def create_dataframe(dictionary):
    for key in dictionary:
        average_dict.update({key:[]})
        average_dict[key].append(sum(dictionary[key])/len(dictionary[key]))
    for key in dictionary:
        stdv_dic.update({key:[]})
        stdv_dic[key].append(np.std(dictionary[key]))
    result = pd.DataFrame.from_dict(dictionary)
    for index, row in result.iterrows():
        result.rename(index={index:change_name_index(index)},inplace=True)
    average=pd.DataFrame.from_dict(average_dict)
    average.index=['Average']
    stdv=pd.DataFrame.from_dict(stdv_dic)
    stdv.index=['stdv']
    frames=[result, average, stdv]
    final_result=pd.concat(frames)
    write_csv(final_result)

def write_csv(result):
    result.to_csv('RDF-based-approach-1_Stardog.csv')

In [6]:
conn=stardog.Connection(database_name,**connection_details)

In [2]:
conn.begin()

In [3]:
print(conn.select('select (count(*) as ?c) where{?n ?m ?l.}'))

In [9]:
Start_Proces=datetime.now()

In [ ]:
for i in range(2):
    print('round '+ str(i+1)+ ' started')
    query1="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?id  ?lbl  
        WHERE{  
            ?n1 rdfs:label ?lbl.  
            ?n1 ns1:name 'XU JIE'.  
            FILTER NOT EXISTS{?n1 ns1:PointsTo ?n.}  
            OPTIONAL{?n1 ns1:node_id ?id.} 
        }        
    """
    calculate_time(query1,"query1")

    query2="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?lbl ?country 
        WHERE{ 
            ?n rdfs:label ?lbl. 
            ?n ns1:name 'EL PORTADOR'. 
            ?n ns1:countries ?country. 
            FILTER NOT EXISTS{ ?n ns1:PointsTo ?n1.} 
            FILTER(?country!='')   
        }         
        """
    calculate_time(query2,"query2")

    query3="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT  DISTINCT ?lbl  
       WHERE{   
            ?n rdfs:label ?lbl.  
            FILTER NOT EXISTS{?n ns1:PointsTo ?n1.}  
        }              
        """
    calculate_time(query3,"query3")
    
    query4="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT DISTINCT ?lbl   
        WHERE{     
            ?r1 rdfs:label ?lbl.  
            ?r1 ns1:PointsTo ?n1. 
        }          
        """
    calculate_time(query4,"query4")    

    query5="""
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX ns1: <http://example.org/>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            SELECT ?name ?lbl  
            WHERE{     
                ?n1 ns1:name 'ORION HOUSE SERVICES (HK) LIMITED'. 
                ?n1 ns1:InRelationTo ?r1. 
                ?r1 rdfs:label 'intermediary_of'. 
                ?r1 ns1:PointsTo ?n2. 
                ?n2 rdfs:label ?lbl. 
                OPTIONAL{?n2 ns1:name ?name.} 
            }                  
        """
    calculate_time(query5,"query5")
    
    query6="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?lbl1 ?name  ?link ?lbl2 
        WHERE{   
              { ?n1 rdfs:label ?lbl1.  
                ?n1 ns1:InRelationTo ?r1.  
                ?r1 rdfs:label 'officer_of'.  
                ?r1 ns1:PointsTo ?n2. 
                ?n2 rdfs:label ?lbl2. 
                ?n1 ns1:name 'BEARER'. 
                OPTIONAL { ?n2 ns1:name ?name. } 
                OPTIONAL { ?r1 ns1:link ?link. }
            }  
            UNION {  
                ?n1 rdfs:label ?lbl1.  
                ?n1 ns1:InRelationTo ?r1.  
                ?r1 rdfs:label 'officer_of'.  
                ?r1 ns1:PointsTo ?n2.  
                ?n2 rdfs:label ?lbl2. 
                ?n1 ns1:name 'The Bearer'.  
                OPTIONAL { ?n2 ns1:name ?name. }  
                OPTIONAL { ?r1 ns1:link ?link. } 
          }  
        }           
        """
    calculate_time(query6,"query6")

    query7=""" 
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?lbl1 ?link ?lbl2  ?entity_name 
        WHERE{   
            {?n1 rdfs:label ?lbl1.  
            ?n1 ns1:InRelationTo ?r1.  
            ?r1 ns1:PointsTo ?n2.  
            ?n2 rdfs:label ?lbl2.  
            ?r1 ns1:end_date '11-APR-2014'.  
            ?n2 ns1:status 'Active'. 
            ?n1 ns1:name 'THE BEARER'. 
            OPTIONAL { ?r1 ns1:link ?link. }   
            OPTIONAL { ?n2 ns1:name ?entity_name. }  
            }  
            UNION  
            {  
            ?n1 rdfs:label ?lbl1.  
            ?n1 ns1:InRelationTo ?r1.  
            ?r1 ns1:PointsTo ?n2.  
            ?n2 rdfs:label ?lbl2.  
            ?r1 ns1:end_date '29-DEC-2009'.  
            ?n2 ns1:status 'Defaulted'. 
            ?n1 ns1:name 'THE BEARER'. 
            OPTIONAL { ?r1 ns1:link ?link. }  
            OPTIONAL { ?n2 ns1:name ?entity_name. }  
        }  
        }               
        """
    calculate_time(query7,"query7")

    query8="""
            PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            PREFIX ns1: <http://example.org/>
            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
            SELECT ?lbl1 ?name ?link ?address  
            WHERE{  
                 ?n1 rdfs:label ?lbl1.  
                ?n1 ns1:InRelationTo ?r1.  
                 ?r1 rdfs:label 'registered_address'.  
                 ?r1 ns1:PointsTo ?n3.  
                OPTIONAL { ?n1 ns1:name ?name. }  
                 OPTIONAL { ?r1 ns1:link ?link. }  
                 OPTIONAL { ?n3 ns1:address ?address. }  
                 }                    
        """
    calculate_time(query8,"query8")
    
    query9="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?lbl ?name ?link1 ?first_address ?link2 ?second_address    
        WHERE { 
             ?n1 rdfs:label ?lbl. 
              ?n1 ns1:InRelationTo ?r1.  
              ?r1 rdfs:label 'registered_address'.  
              ?r1 ns1:PointsTo ?n2.  
             ?n1 ns1:InRelationTo ?r2.    
              ?r2 rdfs:label 'registered_address'.  
              ?r2 ns1:PointsTo ?n3.   
              FILTER(?n2 != ?n3) 
              OPTIONAL { ?n1 ns1:name ?name. } 
             OPTIONAL { ?r1 ns1:link ?link1. }  
              OPTIONAL { ?n2 ns1:address ?first_address. }  
              OPTIONAL { ?r2 ns1:link ?link2. }  
              OPTIONAL { ?n3 ns1:address ?second_address. }  
        }
        """
    calculate_time(query9,"query9")

    query10="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?link1 ?link2 ?address 
        WHERE{ 
        ?n1 rdfs:label 'officer'. 
        ?n1 ns1:InRelationTo ?r1. 
        ?r1 rdfs:label 'registered_address'. 
        ?r1 ns1:PointsTo ?n3. 
        ?n2 rdfs:label 'entity'. 
        ?n2 ns1:InRelationTo ?r2. 
        ?r2 rdfs:label 'registered_address'. 
        ?r2 ns1:PointsTo ?n3. 
               OPTIONAL { ?r1 ns1:link ?link1. } 
               OPTIONAL { ?r2 ns1:link ?link2. } 
               OPTIONAL { ?n3 ns1:address ?address. } 
        }            
        """
    calculate_time(query10,"query10")

    query11="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?name ?valid ?sourceID  
        WHERE{  
            ?n1 rdfs:label 'intermediary'. 
             ?n2 rdfs:label 'entity'. 
             ?n3 rdfs:label 'address'. 
             ?n4 rdfs:label 'officer'. 
             ?n1 ns1:InRelationTo ?r1.  
             ?r1 ns1:PointsTo ?n2.  
             ?n2 ns1:InRelationTo ?r2.  
             ?r2 ns1:PointsTo ?n3.  
             ?n4 ns1:InRelationTo ?r3.  
             ?r3 ns1:PointsTo ?n3.  
             OPTIONAL { ?r1 ns1:valid_until ?valid. }  
             OPTIONAL { ?r1 ns1:sourceID ?sourceID. }  
             OPTIONAL { ?n1 ns1:name ?name. }  
        } 
        """
    calculate_time(query11,"query11")
    
    query12="""
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        PREFIX ns1: <http://example.org/>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?officer_name ?link1 ?officer2_name ?link2 ?entity_name ?intermediary_name ?valid_until  ?e_name  
        WHERE{ 
        ?n1 rdfs:label 'officer'. 
        ?n1 ns1:InRelationTo ?r1. 
       ?r1 rdfs:label 'officer_of'. 
        ?r1 ns1:PointsTo ?n2.
        ?n2 rdfs:label 'officer'. 
        ?n2 ns1:InRelationTo ?r2. 
        ?r2 rdfs:label 'registered_address'. 
        ?r2 ns1:PointsTo ?n3.
        ?n3 rdfs:label 'address'. 
        ?n2 ns1:InRelationTo ?r3. 
        ?r3 rdfs:label 'officer_of'. 
        ?r3 ns1:PointsTo ?n4. 
        ?n4 rdfs:label 'entity'. 
            ?n5 rdfs:label 'intermediary'. 
            ?n5 ns1:InRelationTo ?r4. 
            ?r4 rdfs:label 'intermediary_of'. 
            ?r4 ns1:PointsTo ?n4. 
            ?n5 ns1:InRelationTo ?r5. 
            ?r5 rdfs:label 'intermediary_of'. 
            ?r5 ns1:PointsTo ?n6. 
            ?n6 rdfs:label 'entity'. 
            OPTIONAL {?n1 ns1:name ?officer_name. }  
             OPTIONAL {?r1 ns1:link ?link1.}  
             OPTIONAL {?n2 ns1:name ?officer2_name.}  
             OPTIONAL {?r3 ns1:link ?link2.}  
             OPTIONAL {?n4 ns1:name ?entity_name.}  
             OPTIONAL {?n5 ns1:name ?intermediary_name.}  
             OPTIONAL {?r4 ns1:valid_until ?valid_until.}  
             OPTIONAL {?n6 ns1:name ?e_name.}  
            FILTER(?n6!=?n4) 
        } 
        """
    calculate_time(query12,"query12")
    print('round '+ str(i+1)+ ' finished')

In [11]:
End_Process=datetime.now()

In [12]:
Total_Time_Process=End_Process-Start_Proces

In [14]:
create_dataframe(Time_Dictionary)

In [1]:
print(round((Total_Time_Process ).total_seconds()*1000))